# Elasticity with TDNNS elements

In this notebook we briefly discuss the TDNNS (tangential displacement normal-normal stress) method for linear elasticity. It is a mixed formulation, which does not suffer from shear locking.

References: 
* [<a href="https://numa.jku.at/media/filer_public/b7/42/b74263c9-f723-4076-b1b2-c2726126bf32/phd-sinwel.pdf">Sinwel. A new family of mixed finite elements for elasticity. <i>PhD thesis </i>, Linz (2009).</a>]
* [<a href="https://doi.org/10.1142/S0218202511005568">Pechstein, Schöberl. Tangential-Displacement and Normal-Normal-Stress Continuous Mixed Finite Elements for Elasticity. <i>Math. Models Methods Appl. Sci. </i>, (2011).</a>] 
* [<a href="https://doi.org/10.1002/nme.3319">Pechstein, Schöberl. Anisotropic mixed finite elements for elasticity. <i>Int. J. Numer. Meth. Engng </i>, (2012).</a>] 
* [<a href="https://doi.org/10.1007/s00211-017-0933-3">Pechstein, Schöberl. An analysis of the TDNNS method using natural norms. <i>Numer. Math. </i>, (2018).</a>] 
* [<a href="https://doi.org/10.1016/j.cma.2021.113857">Neunteufel, Pechstein, Schöberl. Three-field mixed finite element methods for nonlinear elasticity. <i>Comput. Methods Appl. Mech. Eng. </i>, (2021).</a>] 


The elasticity problem: Find $u\in H^1_{\Gamma_D}(\Omega,\mathbb{R}^2)$ such that for all $v\in H^1_{\Gamma_D}(\Omega,\mathbb{R}^2)$
\begin{align*}
\int_{\Omega}\mathbb{C}\varepsilon(u):\varepsilon(v)\,dx = \int_{\Omega} f\cdot v\,dx
\end{align*}
can be rewritten as a mixed saddle-point problem by using the stress $\sigma=\mathbb{C}\varepsilon(u)$ as an additional unknown. To this end, we invert the stress-strain relation ($\mathbb{C}^{-1}$ is called the compliance tensor)
\begin{align*}
\sigma =\mathbb{C}\varepsilon(u)= 2\mu\varepsilon(u)+\lambda\,\mathrm{tr}(\varepsilon(u))I_{d\times d},\qquad \varepsilon(u)=\mathbb{C}^{-1}\sigma = \frac{1}{2\mu}\mathrm{dev}(\sigma)+\frac{1}{d(d\lambda+2\mu)}\mathrm{tr}(\sigma)I_{d\times d},
\end{align*}
where $d$ is is the spatial dimension of the domain $\Omega\subset\mathbb{R}^d$, $\mathrm{tr}(A)$ denotes the trace and $\mathrm{dev}(A)=A-\frac{1}{d}\mathrm{tr}(A)I_{d\times d}$ the deviatoric part of the matrix $A$. With this we obtain: Find $(\sigma,u)$ such that for all $(\tau,v)$
\begin{align*}
\begin{array}{cccl}
&\int_{\Omega}\mathbb{C}^{-1}\sigma:\tau\,dx&-\langle \varepsilon(u),\tau\rangle &=&0\\
&-\langle \varepsilon(v),\sigma\rangle &&=&-\int_{\Omega}f\cdot v\,dx
\end{array}
\end{align*}
Depending on the regularity of $\sigma$ and $v$ the duality pairing $\langle\cdot,\cdot\rangle$ has to be interpreted differently. Setting $u\in H^1(\Omega,\mathbb{R}^d)$ and $\sigma\in L^2_{\mathrm{sym}}(\Omega,\mathbb{R}^{d\times d})$ would yield the primal mixed formulation, which is equivalent to the standard formulation and therefore less interesting. Moving all the regularity to the stress by saying $u\in L^2(\Omega,\mathbb{R}^d)$ and $\sigma \in H(\mathrm{div},\mathbb{R}_{\mathrm{sym}}^{d\times d})$, i.e. $\mathrm{div}(\sigma)$ taken row-wise is $L^2$-integrable gives the dual mixed problem which is not equivalent to the first one. To construct conforming finite elements for $H(\mathrm{div},\mathbb{R}_{\mathrm{sym}}^{d\times d})$ is tedious because of the symmetry requirement and that the normal component $\sigma n$ has to be continuous over element interfaces. The construction requires high polynomial degree and (too restrictive) full continuity at vertices is forced. Thus, people investigated so-called weak-symmetric elements, where copies of $H(\mathrm{div})$ conforming elements are taken and the symmetry is enforced weakly by a Lagrange multiplier.

For the TDNNS (tangential displacement normal-normal stress) method we split the regularity between $\sigma$ and $u$, i.e. we use $\sigma\in H(\mathrm{divdiv})$ and $u\in H(\mathrm{curl})$. For $H(\mathrm{curl})$ we have standard Nedelec finite elements. For $\sigma\in H(\mathrm{divdiv})$ we can construct finite elements, which are symmetric, but only the normal-normal component is required to be continuous over element facets, which is way less restrictive. The price we have to pay is that we obtain a duality pairing between a functional and a function, which reads
\begin{align*}
\langle \varepsilon(u),\boldsymbol{\sigma}\rangle &=\sum_{T\in\mathcal{T}}\left(\int_T\boldsymbol{\sigma}:\varepsilon(u)\,dx -\int_{\partial T}\boldsymbol{\sigma}_{nn}u_n\,ds\right)\\
&=-\sum_{T\in\mathcal{T}}\left(\int_T\mathrm{div}\boldsymbol{\sigma}\cdot u\,dx +\int_{\partial T}\boldsymbol{\sigma}_{nt}u_t\,ds\right)=-\langle u, \mathrm{div} \boldsymbol{\sigma}\rangle
\end{align*}

In [ ]:
from ngsolve import *
from ngsolve.meshes import MakeStructured2DMesh
from ngsolve.webgui import Draw

In [ ]:
mapping = lambda x,y : (x,0.1*y)
mesh = MakeStructured2DMesh(quads=False,nx=20,ny=2,mapping=mapping)
Draw(mesh)

# Lame parameter
mu  = 1e3
lam = 5

force = CF( (0,-10) )

In [ ]:
def MaterialInv(mat, mu, lam, d=2):
    return 1/(2*mu)*Deviator(mat)+1/(d*(d*lam+2*mu))*Trace(mat)*Id(d)

def SolveTDNNS(order, mesh):
    # clamped at the left, free else
    fesU = HCurl(mesh, order=order, dirichlet="left")
    fesS = HDivDiv(mesh, order=order, dirichlet="right|top|bottom")
    X = fesS*fesU
    (sigma,u),(tau,v) = X.TnT()
            
    n = specialcf.normal(2)  
    def tang(u): return u-(u*n)*n
            
    a = BilinearForm(X, symmetric=True)
    # small regularization -mu*1e-10*u*v to allow for sparsecholesky solver
    a += (InnerProduct(MaterialInv(sigma,mu,lam),tau) + div(sigma)*v + div(tau)*u - mu*1e-10*u*v)*dx
    a += (-(sigma*n)*tang(v) - (tau*n)*tang(u))*dx(element_boundary=True)
    a.Assemble()
            
    f = LinearForm(X)
    f += -force*v*dx
    f.Assemble()
            
    gfsol = GridFunction(X)
    gfsigma, gfu = gfsol.components
    
    gfsol.vec.data = a.mat.Inverse(X.FreeDofs(), inverse="sparsecholesky") * f.vec
    
    return gfu, gfsigma

In [ ]:
gfu, gfsigma = SolveTDNNS(3, mesh)

Draw(gfu, mesh, "u", deformation=True)
Draw(gfsigma[0,0], mesh, "sigmaxx", deformation=BoundaryFromVolumeCF(gfu))

A mathematical reason why the TDNNS method does not suffer from shear locking is that during the analysis Korn's inequality is never used and one can prove anisotropic error estimates (split into the directions) when using quadrilateral shapes. From an engineering point of view one can argue that the displacement is not fully continuous and therefore more flexible by allowing small gaps, which vanish when the mesh becomes finer. This gaps relax the system's energy such that high stresses and resulting locking is prevented.

In [ ]:
mesh = MakeStructured2DMesh(quads=False,nx=3,ny=1,mapping=mapping)
gfu,gfsigma = SolveTDNNS(1, mesh=mesh)

Draw(gfu, mesh, "u", deformation=True)

One can use hybridization techniques to eliminate the stress locally and obtain a positive definite minimization problem instead of a saddle-point problem. By using a three-field Hu-Washizu formulation using a strain field as an additional unknown, the TDNNS method can be extended to nonlinear material laws.